<a href="https://colab.research.google.com/github/erika024/data-science/blob/main/RAG_GL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain
!pip install torch
!pip install sentence_transformers
!pip install faiss-cpu
!pip install huggingface-hub
!pip install pypdf
!pip -q install accelerate
!pip install llama-cpp-python
!pip -q install git+https://github.com/huggingface/transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.4/116.4 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 17.6 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Us

In [2]:
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import LlamaCpp
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
# from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#  ADD PATH
# loader = TextLoader('/content/drive/MyDrive/exec ed GL/RAG/AAPL-MDA.txt')
# data = loader.load()

# import whole directory...
loader2 = DirectoryLoader('/content/drive/MyDrive/RAG/MDA Data', glob="*.txt", loader_cls=TextLoader)
data2 = loader2.load()
print(data2[0])


page_content=' Item 7. Management\'s Discussion and Analysis of Financial Condition and Results of Operations See "Management\'s Discussion and Analysis" contained in the Registrant\'s 2002 Annual Report to Shareholders, which information is incorporated herein by reference. Item 7A.' metadata={'source': '/content/drive/MyDrive/RAG/MDA Data/ADP.txt'}


In [5]:
print(data2)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [6]:
#Step 05: Split the Extracted Data into Text Chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=50)

text_chunks = text_splitter.split_documents(data2)


In [7]:
len(text_chunks)

12597

In [8]:
#get the third chunk
text_chunks[2].page_content

"data-related operations, accounts receivable, accounts payable, fixed assets, payroll, general accounting, financial accounting consolidation, cash management, human resources, tax, legal and real estate. Under these agreements, we reimbursed Eaton for its direct and indirect costs of providing these services until completion of the divestiture, and thereafter, for a limited time, we reimbursed Eaton for its costs plus an additional fee. The transition periods covered by these agreements generally expired on December 29, 2001. The agreements did not necessarily reflect the costs of obtaining these services from unrelated third parties or of providing the applicable services in-house. However, management believed that purchasing these services from Eaton provided an efficient means of obtaining these services during the transition period. Basis of Presentation On June 30, 2000, Eaton substantially completed the transfer of all the assets and related liabilities of its semiconductor equ

In [9]:
len(text_chunks[2].page_content)

1996

In [10]:
#Step 06:Downlaod the Embeddings
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
#Step 08: Create Embeddings for each of the Text Chunk
vector_store = FAISS.from_documents(text_chunks, embedding=embeddings)

In [12]:
!pip install -q torch datasets
!pip install -q accelerate==0.21.0 \
                peft==0.4.0 \
                bitsandbytes==0.40.2 \
                transformers \
                trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 8.9 MB/s eta 0:00:00


In [13]:
import transformers
from transformers import AutoTokenizer,BitsAndBytesConfig,AutoModelForCausalLM
# from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

In [14]:
from transformers import models

In [15]:
!pip install huggingface_hub -q

In [16]:
from huggingface_hub import hf_hub_download

In [17]:
model_name_or_path = "TheBloke/Llama-2-7B-chat-GGUF"
model_basename = "llama-2-7b-chat.Q5_K_M.gguf" # the model is in gguf format

In [18]:
model_path = hf_hub_download(
    repo_id=model_name_or_path,
    filename=model_basename
    )

llama-2-7b-chat.Q5_K_M.gguf:   0%|          | 0.00/4.78G [00:00<?, ?B/s]

In [19]:
sys_prompt= """
you are a helpful assistant who can go through a document and answer questions
"""

In [20]:
sys_prompt

'\nyou are a helpful assistant who can go through a document and answer questions\n'

In [21]:
#Import Model
llm = LlamaCpp(
    prompt= sys_prompt,
    streaming = True,
    model_path=model_path,
    temperature=0.75,
    top_p=1,
    verbose=True,
    n_ctx=4096
)

/usr/local/lib/python3.10/dist-packages/langchain_core/utils/utils.py:159: UserWarning: WARNING! prompt is not default parameter.
                prompt was transferred to model_kwargs.
                Please confirm that prompt is what you intended.
  warnings.warn(
llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /root/.cache/huggingface/hub/models--TheBloke--Llama-2-7B-chat-GGUF/snapshots/191239b3e26b2882fb562ffccdd1cf0f65402adb/llama-2-7b-chat.Q5_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              =

In [22]:
agent = RetrievalQA.from_chain_type(llm=llm, verbose=True, chain_type="stuff", retriever=vector_store.as_retriever(search_kwargs={"k": 2}),chain_type_kwargs = {"verbose":True})

In [23]:
# The year-over-year growth in R&D expense in 2020 was driven primarily by increases in headcount-related expenses.
query ="What drove research and development expense growth in 2020?"

In [24]:
output = agent.run(query)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

were 19.0%, 15.0% and 14.4% for fiscal year 2002, 2001 and 2000, respectively. The increase in R&D expenses as a percentage of revenues in fiscal 2002 compared to fiscal 2001 is primarily the result of lower sales volume. The decrease in R&D expenses in absolute dollars in fiscal 2002 compared to fiscal 2001 results from controlled spending on general R&D programs, consistent with the downturn in the business cycle. The increase in R&D expenses in fiscal 2001 compared to fiscal 2000 was driven by expanded levels of investment in advanced etch and CMP/Clean applications, including developing the technology necessary to support smaller geometries, new insulating materials, copper-based


llama_print_timings:        load time =    6917.78 ms
llama_print_timings:      sample time =      73.76 ms /   118 runs   (    0.63 ms per token,  1599.76 tokens per second)
llama_print_timings: prompt eval time =  780840.97 ms /  1204 tokens (  648.54 ms per token,     1.54 tokens per second)
llama_print_timings:        eval time =   99593.18 ms /   117 runs   (  851.22 ms per token,     1.17 tokens per second)
llama_print_timings:       total time =  881558.23 ms /  1321 tokens



> Finished chain.

> Finished chain.

> Finished chain.


In [25]:
output

" Based on the provided financial statements for 2002, 2001, and 2000, it appears that the increase in research and development (R&D) expenses in 2002 compared to 2001 was primarily driven by increased engineering staffing costs and prototype fabrication costs. Additionally, the statement mentions that R&D expenses in 2002 were also higher due to new process development costs.\nUnhelpful Answer: I'm not sure, I don't know the answer."